In [38]:
import pandas as pd
import os
#collecting contracts name
dataset='smartbugs/data_analysis/'
results_path='../results/'
contracts_path=os.path.join(results_path,dataset,'contracts.csv')
storage_path=os.path.join(results_path,dataset,'manual_analysis_random_samples/')
patches_path=os.path.join(results_path,dataset,'all_patches_stats.csv')


patches=pd.read_csv(patches_path)


In [39]:
# Generate 10 random samples per source code tool
source_code_tools = [
    'sGuard',
    'sGuardPlus',
    'SmartFix',
    "SolGPT",
    'TIPS',
]



for tool in source_code_tools:
    print(tool)
    # Fixed the syntax error: use parentheses around each condition
    patches_tool = patches[(patches['Tool'] == tool) & (patches['mitigates'] == 'yes')]
    
    # Get 10 random samples or all if less than 10 are available
    if len(patches_tool) > 10:
        patches_sample = patches_tool.sample(10, random_state=42)
    else:
        patches_sample = patches_tool
    
    # Save the sampled patches to CSV
    patches_sample.to_csv(os.path.join(storage_path, tool + '_patches.csv'), index=False)
    
    # Print contract names for verification
    print(patches_sample['Patch'].values)
    print('------------------------------------')

sGuard
['mycontract.sol' 'reentrance.sol' 'simple_dao.sol'
 '0x7541b76cb60f4c60af330c208b0623b7f54bf615.sol'
 '0xcead721ef5b11f1a7b530171aab69b16c5e66b6e.sol'
 '0x7a8721a9d64c74da899424c1b52acbf58ddc9782.sol' 'phishable.sol'
 'reentrancy_simple.sol' '0x01f8c4e3fa3edeb29e514cba738d87ce8c091d3f.sol'
 '0xbaf51e761510c1a11bf48dd87c0307ac8a8c8a4f.sol']
------------------------------------
sGuardPlus
['0x7541b76cb60f4c60af330c208b0623b7f54bf615.sol'
 '0xf015c35649c82f5467c9c74b7f28ee67665aad68.sol'
 '0x4e73b32ed6c35f570686b89848e5f39f20ecc106.sol' 'reentrancy_dao.sol'
 '0x7b368c4e805c3870b6c49a3f1f49f69af8662cf3.sol' 'phishable.sol'
 '0x52d2e0f9b01101a59b38a3d05c80b7618aeed984.sol'
 'overflow_simple_add.sol' 'integer_overflow_mul.sol'
 '0xb7c5c5aa4d42967efe906e1b66cb8df9cebf04f7.sol']
------------------------------------
SmartFix
['0x8c7777c45481dba411450c228cb692ac3d550344.sol' 'phishable.sol'
 '0x941d225236464a25eb18076df7da6a91d0f95e9e.sol'
 'wallet_02_refund_nosub.sol'
 '0xb93430ce38ac4a

In [40]:
vulnerabilities = patches['Category'].unique()
binary_tools = [ 'Elysium', 'SmartShield']

for vuln in vulnerabilities:
    print(vuln)
    # remove the patches from binary tools
    patches = patches[~patches['Tool'].isin(binary_tools)]
    patches_population = patches[(patches['Category'] == vuln) & (patches['mitigates'] == 'yes')]
    
    # Get 10 random samples or all if less than 10 are available
    if len(patches_population) > 10:
        patches_sample = patches_population.sample(10, random_state=42)
    else:
        patches_sample = patches_population
    
    # Save the sampled patches to CSV
    patches_sample.to_csv(os.path.join(storage_path, vuln + '_patches.csv'), index=False)
    
    # Print contract names for verification
    print(patches_sample['Patch'].values)
    print('------------------------------------')

access_control
['mycontract.sol' 'simple_suicide.sol'
 'incorrect_constructor_name2_4round.sol'
 'wallet_02_refund_nosub_2round.sol' 'simple_suicide.sol'
 'mycontract_2round.sol' 'proxy_4round.sol'
 'incorrect_constructor_name1_4round.sol' 'wallet_02_refund_nosub.sol'
 'mycontract_3round.sol']
------------------------------------
arithmetic
['tokensalechallenge.sol' 'integer_overflow_mul.sol'
 'integer_overflow_mul_4round.sol' 'integer_overflow_add.sol'
 'token_1round.sol'
 'integer_overflow_multitx_multifunc_feasible_3round.sol'
 'integer_overflow_add_3round.sol' 'integer_overflow_mul_2round.sol'
 'integer_overflow_mapping_sym_1_1round.sol' 'timelock.sol']
------------------------------------
bad_randomness
['old_blockhash_3round.sol' 'old_blockhash_1round.sol'
 'old_blockhash_2round.sol' 'blackjack_3round.sol' 'etheraffle_3round.sol'
 'etheraffle_4round.sol' 'etheraffle_1round.sol']
------------------------------------
denial_of_service
[]
------------------------------------
front_r

In [41]:
# add the diff path to the patches

for tool in source_code_tools:
    # read the patches
    patches_tool = pd.read_csv(os.path.join(storage_path, tool + '_patches.csv'))
    for patch_path in patches_tool['patch_link'].values:
        diff_path = patch_path.replace(".sol", ".diff")
        # add diff path to patches_tool in the same row as patch_path
        patches_tool.loc[patches_tool['patch_link'] == patch_path, 'diff_link'] = diff_path
        # order the columns
        patches_tool = patches_tool[['Patch', 'Original', 'Category', 'Tool', 'DIFF', 'COMP', 'Detected', 'Fixed', 'Consistent', 'sanity_check', 'mitigates', 'patch_link', 'original_link', 'diff_link', 'exploit_link']]
    # updat csv
    patches_tool.to_csv(os.path.join(storage_path, tool + '_patches.csv'), index=False)

for vuln in vulnerabilities:
    # read the patches
    patched_vuln = pd.read_csv(os.path.join(storage_path, vuln + '_patches.csv'))
    for patch_path in patched_vuln['patch_link'].values:
        diff_path = patch_path.replace(".sol", ".diff")
        # add diff path to patched_vuln in the same row as patch_path
        patched_vuln.loc[patched_vuln['patch_link'] == patch_path, 'diff_link'] = diff_path
        # order the columns
        patched_vuln = patched_vuln[['Patch', 'Original', 'Category', 'Tool', 'DIFF', 'COMP', 'Detected', 'Fixed', 'Consistent', 'sanity_check', 'mitigates', 'patch_link', 'original_link', 'diff_link', 'exploit_link']]
    # updat csv
    patched_vuln.to_csv(os.path.join(storage_path, vuln + '_patches.csv'), index=False)



Code used to generate .diff files. Should be run after formatting with prettier-solidity.
** diffs already included in the repo, no need to run **

In [42]:
import difflib

def read_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.readlines()

def diff_files(file1, file2, file1_path, file2_path):
    formatted1 = read_file(file1)
    formatted2 = read_file(file2)

    diff = difflib.unified_diff(
        formatted1, formatted2,
        fromfile=file1_path, tofile=file2_path,
        lineterm=''
    )

    return '\n'.join(diff)

def patch_path(path):
    github_prefix = "https://github.com/ASSERT-KTH/RepairComp/blob/main/results/"

    dir_prefix = "../results/"

    path = path.replace(github_prefix, dir_prefix)
    return path

def og_path(path):
    github_prefix = "https://github.com/smartbugs/smartbugs-curated/tree/main/"
    dir_prefix = "../../smartbugs-curated/"
    path = path.replace(github_prefix, dir_prefix)
    return path

def generate_diffs():
    for vuln in vulnerabilities:
        # read the patches
        patches_vuln = pd.read_csv(os.path.join(storage_path, vuln + '_patches.csv'))

        # iterate patch_link and original_link from the patches using zip
        for patch_link, original_link in zip(patches_vuln['patch_link'], patches_vuln['original_link']):
            # patch_path and og_path are functions that replace the prefix of the path
            patch = patch_path(patch_link)
            original = og_path(original_link)

            #  check if patch and original exist
            if not os.path.exists(patch):
                print(f"Patch file does not exist: {patch}")
                # continue
            if not os.path.exists(original):
                print(f"Original file does not exist: {original}")
                # continue

            # diff_files is a function that returns the diff between two files
            diff = diff_files(original, patch, original_link, patch_link)

            filename = patch.replace('.sol', '.diff')
            print("Filename: ", filename)

            # Save the diff to a file
            with open(filename, 'w') as f:
                f.write(diff)

In [43]:
def generate_diffs_for_all(force_regenerate=True):
    """
    Generate diff files for all source code patches in all_patches_stats.csv 
    Places diff files in the same directory as each patch file.
    Uses the existing patch_path, og_path, and diff_files functions.
    
    Args:
        force_regenerate (bool): If True, regenerate even if diff file exists
    """
    all_patches_stats = pd.read_csv(patches_path)


    print(f"Processing {len(all_patches_stats)} patches for diff generation...")
    if force_regenerate:
        print("⚠️  Force regeneration enabled - will replace existing diff files")
    
    stats = {
        'total': len(all_patches_stats),
        'generated': 0,
        'replaced': 0,
        'skipped_existing': 0,
        'skipped_missing_files': 0,
        'errors': 0
    }
    
    # Iterate through all patches in all_patches_stats
    for idx, row in all_patches_stats.iterrows():
        patch_link = row['patch_link']
        original_link = row['original_link']
        
        try:
            # Convert GitHub URLs to local paths using existing functions
            patch = patch_path(patch_link)
            original = og_path(original_link)
            
            # Skip bytecode files (.bin, .hex) since we don't have source code to compare
            patch_basename = os.path.basename(patch)
            patch_ext = os.path.splitext(patch_basename)[1].lower()
            
            if patch_ext in ['.bin', '.hex']:
                print(f"Skipping bytecode file: {patch_basename}")
                stats['skipped_missing_files'] += 1
                continue
            
            # Create diff filename in the same directory as the patch file
            patch_dir = os.path.dirname(patch)
            
            # Remove extension and add .diff
            diff_basename = os.path.splitext(patch_basename)[0] + '.diff'
            diff_filename = os.path.join(patch_dir, diff_basename)
            
            # Check if diff already exists
            if os.path.exists(diff_filename):
                if force_regenerate:
                    print(f"Replacing existing diff: {diff_basename}")
                    stats['replaced'] += 1
                else:
                    print(f"Diff already exists, skipping: {diff_filename}")
                    stats['skipped_existing'] += 1
                    continue
            
            # Check if source files exist
            missing_files = []
            if not os.path.exists(patch):
                missing_files.append(f"Patch: {patch}")
            if not os.path.exists(original):
                missing_files.append(f"Original: {original}")
            
            if missing_files:
                print(f"Missing files for row {idx}:")
                for missing in missing_files:
                    print(f"  - {missing}")
                stats['skipped_missing_files'] += 1
                continue
            
            # Generate the diff using existing function (fix parameter order)
            print(f"Generating diff for: {patch_basename} -> {diff_basename}")
            diff_content = diff_files(original, patch, original_link, patch_link)
            
            # Ensure patch directory exists (should already exist if patch file exists)
            if not os.path.exists(patch_dir):
                os.makedirs(patch_dir, exist_ok=True)
            
            # Save the diff to file in the same directory as the patch
            with open(diff_filename, 'w', encoding='utf-8') as f:
                f.write(diff_content)
            
            if not os.path.exists(diff_filename) or stats['replaced'] == 0:
                stats['generated'] += 1
            
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            print(f"  Patch link: {patch_link}")
            print(f"  Original link: {original_link}")
            stats['errors'] += 1
            continue
    
    # Print summary statistics
    print(f"\n=== Diff Generation Summary ===")
    print(f"Total patches processed: {stats['total']}")
    print(f"New diffs generated: {stats['generated']}")
    print(f"Existing diffs replaced: {stats['replaced']}")
    print(f"Already existing (skipped): {stats['skipped_existing']}")
    print(f"Missing source files (skipped): {stats['skipped_missing_files']}")
    print(f"Errors encountered: {stats['errors']}")
    
    total_created = stats['generated'] + stats['replaced']
    if total_created > 0:
        print(f"\n✅ Successfully created/replaced {total_created} diff files!")
    else:
        print(f"\n⚠️  No diff files were generated or replaced.")




In [44]:
# Generate all diffs, replacing existing ones (recommended for your situation)
#generate_diffs_for_all(force_regenerate=True)